In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

from spatial_split import *
from utils import *


24/05/28 16:33:02 WARN Utils: Your hostname, yaozys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.30.193.7 instead (on interface en0)
24/05/28 16:33:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/28 16:33:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/28 16:33:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 1. Load Data

In [2]:
from sklearn.datasets import make_blobs

# Generate random dataset with 3 dimensions
n_samples = 100  # Number of samples
n_features = 3  # Number of dimensions
centers = 10  # Number of clusters
random_state = 42  # Random state for reproducibility

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)
X = X.tolist()

In [3]:
pindex = list(range(len(X)))
pindex

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

# 2. Set parameter

In [4]:
# set parameters
min_pts = 5
eps = 2
n_pa_each_dim = [3 for i in range(n_features)]

n_features = len(X[0])


# 3. Parallelize Data

In [27]:
rdd = sc.parallelize(X)

# 4. Construct grid cells and locate points into cell

In [28]:
# construct the grid and locate the points
min_max_bounds = get_minmax_by_data(X)
n_grid_each_dim = cal_grid_num(min_max_bounds, eps)
grid_bin_bounds = get_bin_bounds(min_max_bounds, n_grid_each_dim)
grid_rdd = rdd.map(lambda x: (find_location_id(x, grid_bin_bounds, n_grid_each_dim), x))
grid_rdd = grid_rdd.groupByKey().mapValues(list)
grid_rdd.cache()
grid_rdd.take(10)

[((8, 14, 3),
  [[-0.290283109174704, 5.984511095595305, -6.629223876653399],
   [-0.4164968413960098, 5.902578923433743, -6.606741233991601],
   [-0.028998218638255913, 6.06053471382532, -6.69943395209346]]),
 ((8, 6, 11), [[-0.8460643599490254, -3.6616312451269524, 2.752105580753638]]),
 ((8, 11, 0),
  [[-0.025577988321223488, 2.1724577297292917, -9.201134799967729]]),
 ((15, 4, 2), [[8.545645798662383, -5.9986059024373475, -7.117236820215477]]),
 ((9, 13, 3), [[0.7538109882723547, 4.27337784989964, -6.446568843529789]]),
 ((2, 15, 10), [[-7.922925638933936, 7.652274025158388, 1.4925400310971368]]),
 ((1, 6, 6), [[-9.23526537361677, -3.970652714526209, -3.334549598894554]]),
 ((7, 17, 13), [[-1.8975213342118824, 10.045285650694272, 5.5711589553443]]),
 ((9, 11, 1),
  [[1.1842886437054831, 2.155590898117459, -8.258129626761084],
   [1.0319823733954943, 2.458661642674315, -9.091893339564193]]),
 ((5, 17, 11), [[-3.7300412730237724, 9.223149723203077, 2.6802087123483256]])]

In [30]:
grid_rdd.partitioner.partitionFunc

<function pyspark.rdd.portable_hash(x: Hashable) -> int>

# 5. Partition data

In [31]:
# allocate grid into partitions

def add_partition_id(x, grid_bins, n_pa_each_dim, buffer_size):
    buffer_loc_id = find_buffer_location_id(x[0], grid_bins, n_pa_each_dim, buffer_size)
    buffer_loc_id = list(set(buffer_loc_id))
    return [(blid, (x[0], x[1])) for blid in buffer_loc_id]

def get_partitioned_cells(rdd, grid_bins, n_pa_each_dim, buffer_size):
    rdd1 = rdd.flatMap(lambda x: add_partition_id(x, grid_bins, n_pa_each_dim, buffer_size))
    partitioned_rdd = rdd1.groupByKey().mapValues(list)
    # partitioned_rdd = rdd1.sortByKey()
    return partitioned_rdd

# divide the grid into partitions
grid_min_max = np.array(([0 for _ in range(len(n_grid_each_dim))], [i-1 for i in n_grid_each_dim])).T
grid_bins = get_bin_bounds(grid_min_max, n_pa_each_dim)

# buffer size (number of cells to be included in half of the buffer region)
buffer_size = int(np.ceil(eps/cal_grid_side_len(eps, n_features)))
partitioned_rdd = get_partitioned_cells(grid_rdd, grid_bins, n_pa_each_dim, buffer_size)
partitioned_rdd.glom().take(2)

[[((1, 0, 0),
   [((9, 13, 3), [[0.7538109882723547, 4.27337784989964, -6.446568843529789]]),
    ((10, 2, 2),
     [[1.4548994656670846, -7.688120794044457, -7.381866636860483]]),
    ((10, 3, 1),
     [[1.6741623334748645, -6.787866414615768, -8.867678507876839]]),
    ((9, 12, 3),
     [[1.254433058996986, 3.7514314283377708, -6.158309451868389]]),
    ((9, 3, 3),
     [[1.1339521607180933, -7.188839567002485, -6.548846161872382]]),
    ((10, 3, 4), [[1.75349779610322, -6.522514619639524, -5.40221554853443]]),
    ((4, 6, 5),
     [[-5.444668546678066, -3.752125318334681, -3.9336470884612114]]),
    ((9, 2, 3),
     [[0.8668347099347038, -8.075833815231942, -6.067583770881749]]),
    ((9, 12, 0),
     [[0.40201615158101456, 3.1259562730292747, -9.662563134435874]]),
    ((11, 3, 3),
     [[3.3294097125115547, -6.951637312731604, -5.876576695383923]]),
    ((11, 3, 4),
     [[3.511206250406701, -6.915453230261222, -5.315465937461941]]),
    ((10, 2, 3),
     [[2.334805708988366, -7.5

In [39]:
print("Partitioner: ",partitioned_rdd.partitioner.partitionFunc)
print("Partition number: ", partitioned_rdd.getNumPartitions())

Partitioner:  <function portable_hash at 0x7faffe9c2290>
Partition number:  8


# 6. Identify Core point

In [38]:
# find core points in each partition
def find_core_points(pa, eps, min_pts):
    pa = list(pa)
    for p in pa:
        pid = p[0]
        grids = p[1]
        
        points = []
        for grid in grids:
            for p in grid[1]:
                points.append([pid, grid[0], p])
                
        for pid, gid, value in points:
            count = 0
            core_flag = 0
            for _, _, value2 in points:
                if value == value2:
                    continue
                if np.linalg.norm(np.array(value) - np.array(value2)) <= eps:
                    count += 1
                    if count >= min_pts:
                        core_flag = 1
                        break
            yield ((pid, gid), value, core_flag)

points_with_flag = partitioned_rdd.mapPartitions(lambda x: find_core_points(x, eps, min_pts))
points_with_flag.take(10)


[(((1, 0, 0), (9, 13, 3)),
  [0.7538109882723547, 4.27337784989964, -6.446568843529789],
  0),
 (((1, 0, 0), (10, 2, 2)),
  [1.4548994656670846, -7.688120794044457, -7.381866636860483],
  1),
 (((1, 0, 0), (10, 3, 1)),
  [1.6741623334748645, -6.787866414615768, -8.867678507876839],
  0),
 (((1, 0, 0), (9, 12, 3)),
  [1.254433058996986, 3.7514314283377708, -6.158309451868389],
  0),
 (((1, 0, 0), (9, 3, 3)),
  [1.1339521607180933, -7.188839567002485, -6.548846161872382],
  1),
 (((1, 0, 0), (10, 3, 4)),
  [1.75349779610322, -6.522514619639524, -5.40221554853443],
  1),
 (((1, 0, 0), (4, 6, 5)),
  [-5.444668546678066, -3.752125318334681, -3.9336470884612114],
  0),
 (((1, 0, 0), (9, 2, 3)),
  [0.8668347099347038, -8.075833815231942, -6.067583770881749],
  0),
 (((1, 0, 0), (9, 12, 0)),
  [0.40201615158101456, 3.1259562730292747, -9.662563134435874],
  0),
 (((1, 0, 0), (11, 3, 3)),
  [3.3294097125115547, -6.951637312731604, -5.876576695383923],
  0)]

In [37]:
points_with_flag.partitioner

# 7. Find core cells

In [45]:
# find core cells with at least one core point in each partition

def find_core_cells(pa):
    pa = list(pa)
    core_cells = []
    for key, _, flag in pa:
        if flag == 1:
            if key not in core_cells:
                core_cells.append(key)
                yield key
    
core_cells = points_with_flag.mapPartitions(lambda x: find_core_cells(x))
core_cells.take(10)

# adopted to avoid the shuffle    
# core_cells = points_with_flag.map(lambda x: (x[0], x[2]))\
#                               .reduceByKey(lambda x, y: x+y)\
#                                 .filter(lambda x: x[1] > 0)
# core_cells.take(10)

[((1, 0, 0), (10, 2, 2)),
 ((1, 0, 0), (9, 3, 3)),
 ((1, 0, 0), (10, 3, 4)),
 ((1, 0, 0), (10, 2, 3)),
 ((1, 0, 0), (11, 3, 2)),
 ((0, 0, 1), (2, 6, 9)),
 ((0, 0, 1), (4, 6, 8)),
 ((0, 0, 1), (3, 7, 9)),
 ((0, 0, 1), (3, 6, 8)),
 ((0, 0, 1), (4, 6, 9))]

# 8. Find eps-neighbor cell pair

In [10]:
# join points in the core cells. (shuffle)
# repartition for further processing.(since after a shuffle, the partitioning based on the partition id is lost)

core_cell_rdd = core_cells.map(lambda x: (x[0], 1))\
                        .join(points_with_flag.map(lambda x: (x[0], x[1])))\
                            .map(lambda x: (x[0][0], (x[0][1], x[1][1])))
core_cell_rdd = core_cell_rdd.groupByKey().mapValues(list)
core_cell_rdd.glom().take(1)

[[((1, 0, 0),
   [((10, 2, 3), [2.334805708988366, -7.5247469457563945, -6.518713987767533]),
    ((10, 2, 2), [1.4548994656670846, -7.688120794044457, -7.381866636860483]),
    ((9, 3, 3), [1.1339521607180933, -7.188839567002485, -6.548846161872382]),
    ((10, 3, 4), [1.75349779610322, -6.522514619639524, -5.40221554853443]),
    ((11, 3, 2),
     [2.948714811063091, -7.35880142899656, -7.065768569939764])])]]

In [13]:
def get_distance_matrix(points1, points2):
    distance_matrix = np.zeros((len(points1), len(points2)))
    for i in range(len(points1)):
        for j in range(len(points2)):
            distance_matrix[i][j] = np.linalg.norm(np.array(points1[i]) - np.array(points2[j]))
    return distance_matrix

def find_neighbor_cell(pa, eps):
    pa = list(pa)
    for p in pa:
        pid = p[0]
        grids = p[1]
        
        grid_points = {}
        for g in grids:
            if g[0] not in grid_points:
                grid_points[g[0]] = []
            grid_points[g[0]].extend(g[1])
        
        for gid, points in grid_points.items():
            for gid2, points2 in grid_points.items():
                if gid == gid2:
                    continue
                distance_matrix = get_distance_matrix(points, points2)
                isNeighbor = np.sum(distance_matrix <= eps)
                if isNeighbor >= 1:
                    yield (pid, gid, gid2)

neighbor_pairs = core_cell_rdd.mapPartitions(lambda x: find_neighbor_cell(x, eps))
neighbor_pairs.collect()

[((1, 0, 0), (10, 2, 3), (10, 2, 2)),
 ((1, 0, 0), (10, 2, 3), (9, 3, 3)),
 ((1, 0, 0), (10, 2, 3), (10, 3, 4)),
 ((1, 0, 0), (10, 2, 3), (11, 3, 2)),
 ((1, 0, 0), (10, 2, 2), (10, 2, 3)),
 ((1, 0, 0), (10, 2, 2), (9, 3, 3)),
 ((1, 0, 0), (10, 2, 2), (10, 3, 4)),
 ((1, 0, 0), (10, 2, 2), (11, 3, 2)),
 ((1, 0, 0), (9, 3, 3), (10, 2, 3)),
 ((1, 0, 0), (9, 3, 3), (10, 2, 2)),
 ((1, 0, 0), (9, 3, 3), (10, 3, 4)),
 ((1, 0, 0), (9, 3, 3), (11, 3, 2)),
 ((1, 0, 0), (10, 3, 4), (10, 2, 3)),
 ((1, 0, 0), (10, 3, 4), (10, 2, 2)),
 ((1, 0, 0), (10, 3, 4), (9, 3, 3)),
 ((1, 0, 0), (10, 3, 4), (11, 3, 2)),
 ((1, 0, 0), (11, 3, 2), (10, 2, 3)),
 ((1, 0, 0), (11, 3, 2), (10, 2, 2)),
 ((1, 0, 0), (11, 3, 2), (9, 3, 3)),
 ((1, 0, 0), (11, 3, 2), (10, 3, 4)),
 ((1, 2, 0), (8, 14, 3), (9, 11, 1)),
 ((1, 2, 0), (8, 14, 3), (5, 16, 12)),
 ((1, 2, 0), (8, 14, 3), (8, 13, 3)),
 ((1, 2, 0), (8, 14, 3), (8, 15, 2)),
 ((1, 2, 0), (8, 14, 3), (9, 14, 2)),
 ((1, 2, 0), (8, 14, 3), (9, 10, 0)),
 ((1, 2, 0), (8, 14

# 9. Create a graph

In [14]:
from pyspark.sql import SparkSession
from graphframes import GraphFrame
spark = SparkSession.builder.appName("ApproxDBSCAN").getOrCreate()


In [15]:
pos_to_gid, gid_to_pos = grid_index_mapping(n_grid_each_dim)
pos_to_paid, paid_to_pos = grid_index_mapping(n_pa_each_dim)

In [43]:
int(np.prod(n_pa_each_dim))

numpy.int64

In [44]:
v = core_cells.map(lambda x: (pos_to_paid[x[0][0]], pos_to_gid[x[0][1]],)).toDF(["paid", "id"])
v = v.repartition(int(np.prod(n_pa_each_dim)), "paid")
e = neighbor_pairs.map(lambda x: (pos_to_paid[x[0]], pos_to_gid[x[1]], pos_to_gid[x[2]])).toDF(["paid", "src", "dst"])
e = e.repartition(int(np.prod(n_pa_each_dim)), "paid")
g = GraphFrame(v, e)

g.edges.show()

/Users/yaozeyi/anaconda3/envs/py310/lib/python3.10/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+----+----+----+
|paid| src| dst|
+----+----+----+
|  21|3096|3132|
|  21|3096|3115|
|  21|3096|3439|
|  21|3096|3114|
|  21|4287|4936|
|  21|4287|4612|
|  21|4287|4611|
|  21|3132|3096|
|  21|3132|3115|
|  21|3132|3439|
|  21|3132|3114|
|  21|4936|4287|
|  21|4936|4612|
|  21|4936|4611|
|  21|3115|3096|
|  21|3115|3132|
|  21|3115|3439|
|  21|3115|3114|
|  21|3439|3096|
|  21|3439|3132|
+----+----+----+
only showing top 20 rows



In [17]:
# set Checkpoint directory
sc.setCheckpointDir("checkpoints")

# compute connected components
result = g.connectedComponents()
# remove duplicates
result = result.dropDuplicates(['id'])
result.show()


/Users/yaozeyi/anaconda3/envs/py310/lib/python3.10/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+----+---------+
|  id|component|
+----+---------+
|3114|      603|
| 744|      744|
|2790|      603|
|3115|      603|
|1920|      603|
|1412|      744|
|3278|      744|
| 603|      603|
|1938|      603|
|1413|      744|
|2847|      603|
|4612|      744|
|1939|      603|
|3279|      744|
|3096|      603|
|3132|      603|
|4287|      744|
|2829|      603|
|4269|      744|
|4936|      744|
+----+---------+
only showing top 20 rows



# 10. Assign points to clusters

In [24]:
points_with_flag.collect()

[(((1, 0, 0), (9, 13, 3)),
  [0.7538109882723547, 4.27337784989964, -6.446568843529789],
  0),
 (((1, 0, 0), (10, 2, 2)),
  [1.4548994656670846, -7.688120794044457, -7.381866636860483],
  1),
 (((1, 0, 0), (10, 3, 1)),
  [1.6741623334748645, -6.787866414615768, -8.867678507876839],
  0),
 (((1, 0, 0), (9, 12, 3)),
  [1.254433058996986, 3.7514314283377708, -6.158309451868389],
  0),
 (((1, 0, 0), (9, 3, 3)),
  [1.1339521607180933, -7.188839567002485, -6.548846161872382],
  1),
 (((1, 0, 0), (10, 3, 4)),
  [1.75349779610322, -6.522514619639524, -5.40221554853443],
  1),
 (((1, 0, 0), (4, 6, 5)),
  [-5.444668546678066, -3.752125318334681, -3.9336470884612114],
  0),
 (((1, 0, 0), (9, 2, 3)),
  [0.8668347099347038, -8.075833815231942, -6.067583770881749],
  0),
 (((1, 0, 0), (9, 12, 0)),
  [0.40201615158101456, 3.1259562730292747, -9.662563134435874],
  0),
 (((1, 0, 0), (11, 3, 3)),
  [3.3294097125115547, -6.951637312731604, -5.876576695383923],
  0),
 (((1, 0, 0), (11, 3, 4)),
  [3.51120

In [26]:
points_with_flag.partitioner

In [25]:
cluster = result.rdd.map(lambda x: (gid_to_pos[x[0]], x[1]))\
                        .join(points_with_flag.map(lambda x: (x[0][1], (x[1], x[2]))))\
                                .map(lambda x: (x[1][0], x[1][1]))\
                                        .groupByKey().mapValues(list)
cluster.collect()


[(603,
  [([-7.922925638933936, 7.652274025158388, 1.4925400310971368], 1),
   ([-7.922925638933936, 7.652274025158388, 1.4925400310971368], 0),
   ([-2.5226948477906843, 7.956575199242421, 5.4624237483312905], 1),
   ([-2.5226948477906843, 7.956575199242421, 5.4624237483312905], 0),
   ([-2.5226948477906843, 7.956575199242421, 5.4624237483312905], 0),
   ([-2.5226948477906843, 7.956575199242421, 5.4624237483312905], 0),
   ([-2.5226948477906843, 7.956575199242421, 5.4624237483312905], 0),
   ([-2.5226948477906843, 7.956575199242421, 5.4624237483312905], 0),
   ([-3.9877196134201776, 8.294441919803614, 4.1792400652683135], 1),
   ([-3.9877196134201776, 8.294441919803614, 4.1792400652683135], 0),
   ([-3.9877196134201776, 8.294441919803614, 4.1792400652683135], 1),
   ([-3.9877196134201776, 8.294441919803614, 4.1792400652683135], 1),
   ([-3.9877196134201776, 8.294441919803614, 4.1792400652683135], 1),
   ([-3.9877196134201776, 8.294441919803614, 4.1792400652683135], 1),
   ([-3.6601912